## Learning how to access Spotify API using Spotipy wrapper
must provide authentification for each object initiated 

In [1]:
import spotipy
import pandas as pd
import numpy as np
import glob
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
#maybe also consider google trends api

In [3]:
import os
import spotipy
import json
import webbrowser
from time import sleep
import spotipy.util as util
from json.decoder import JSONDecodeError
import requests
from spotipy.oauth2 import SpotifyClientCredentials

#sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


## I need to get the track id for each song and then pass it through the Spotify API to get audio features
1. First I need to load all data frame so that i can get all the songs that i'm interested in

In [246]:
recent_songs = pd.read_pickle('recent_songs_to_spotify.pkl') #collection of songs from kaggle and billboards
#songs_and_artists #there are 41k songs 

#i want to create a dictionary of artist and song from the data frame to be passed through get_track_id function
song_dict = recent_songs.to_dict('records')
#returns a list of dictionaries with the keys:
#dict_keys(['danceability', 'duration', 
# 'energy', 'key', 'key_confidence', 'loudness', 'mode', 
# 'mode_confidence', 'tempo', 'time_signature', 'artist_hotttnesss', 
# 'artist_location', 'Artist', 'genre', 'song_hotttnesss', 'Song', 'year', 'num_weeks'])

#what features are available to me, and which of those do i want to grab?

# i want acousticness, danceability, duration_ms, energy, instrumentalness, key, loudness, mode, speechiness, 
# tempo, time_signature, valence

first_few = song_dict[:10]
first_few
len(song_dict)
recent_songs.drop_duplicates(inplace = True)


In [254]:
np.mean(recent_songs[recent_songs['Artist'] == 'Drake'])

0    3.510204
dtype: float64

In [5]:
outer_merged_df[outer_merged_df['Song'] == 'Sail']

NameError: name 'outer_merged_df' is not defined

In [6]:
#initiatlizing spotipy object with credentials passed int with the credentials manager

#what happens if i create the object before the function?

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

#come back put try excepts in here to account for not being able to find something

def get_track_id(list_of_songs):
    """this function takes a list of song's dictionary and returns the spotify id necessary to retrieve
    the features and analysis that you are interested in.
    """
    track_ids = []
    #for song in list_of_songs:
    unfound_songs = []
    for song in list_of_songs:
        try:
            track_name = song.get('Song')
            #print(track_name)
            artist_name = song.get('Artist')
            #print(artist_name)
            q = f'track:{track_name} artist:{artist_name}'
            results = sp.search(q, type = 'track')
            track_id = results['tracks']['items'][0]['id']
            track_ids.append(track_id)
            #print(track_ids)
        except:
            unfound_songs.append(song)
            pass
    #print(type(results)) #this function returns a dictionary
    unfound_songs_df = pd.DataFrame(unfound_songs)
    unfound_songs_df.to_pickle('unfound_songs.pkl')
    
    return (track_ids)

#get_track_id('justin bieber', 'one less lonely girl')
    #sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [245]:
#%%timeit #each song takes 123 ms to get the track id
#track_id =get_track_id(first_few)
#len(track_id)
#first_few
pd.DataFrame(get_track_features(['0HVv5bEOiI9a0QfgcASXpX'])).to_pickle('Nice_for_what.pkl')
#0HVv5bEOiI9a0QfgcASXpX
#track_id

Nice For What


In [242]:
pd.DataFrame(get_track_features(['0bAkKNCQfWkexHFn7fIKns', 'spotify:track:2Vx8E3K5icPZR7OCklWBXX'])).to_pickle('Drake_for_testing_model.pkl')

In My Feelings


In [239]:
#cool, now that i've verified that the function returns the id for that song, i need to write a function that takes
#that id to feed into spotify to get acousticness, speechiness,  valence from audio_features

def get_track_features(list_of_track_ids):
    array_of_songs = []
    """this function takes a list of each song's dictionary and returns the spotify id necessary to retrieve
    the features and analysis that you are interested in.
    """
    for track_id in list_of_track_ids:
        #print(track_id)
        try:
            features = sp.audio_features(track_id)[0]
            track = sp.track(track_id) #returns a dictionary
            features['artist'] = track['album']['artists'][0]['name']
            features['song'] = track['name']
            print(track['name'])
            #print(track)
            #break
            
            #print(track['album']['artists'][0]['name'], track['album']['name'])
            
            
            #genres = sp.track('59OAfqBKd0FJLn5QzsN3mx')
            artist_id = track.get('album').get('artists')[0]['id']
            artist = sp.artist(artist_id)
            #print(artist)
            features['genre'] = artist['genres']
            #print(features.get('genre'))
            
            features['popularity'] = artist['popularity']
            #print(features)
        except:
            pass
        array_of_songs.append(features)
#     features['num_weeks'] = #i need to grab this info from the initial datafram
    
    return (array_of_songs)


In [9]:
# #get_track_features(track_id)
# def get_artist_genre(array_of_song_dict):
#     """this function takes an array of songs's dict and tries to find the artist's genre and popularity
#     and then returns an array of song dict with those additional keys, yay!
#     """
#     for song in array_of_song_dict:
#         artist_id = song.get('artist_id')
#         print(artist_id)
#         genres = sp.artist(artist_id)
#         print(genres)

In [10]:
def get_songs(list_of_songs):
    """takes a list of songs
    """
    track_ids = get_track_id(list_of_songs) #returns a list of track ids
    track_features = get_track_features(track_ids) #returns a list of song dictionaries
    #print(track_features)
    #more_track_features = get_artist_genre(track_features)
    #pd.DataFrame(track_features)
    #print(type(track_features))
    return pd.DataFrame(track_features)

In [11]:
#%%timeit #2.2 seconds for 10 songs of which 5 were not found on spotify

# song_df = get_songs(song_dict[1000:1050])
#song_df.to_pickle("first_10_of_song_dict.pkl")

In [12]:
# song_df.to_pickle("1000_to_1050_of_song_dict.pkl")
# song_df

## writing a script that creates intervals for me so i dont have to do it manually

In [237]:
# #missing 1350 to 1400
list_of_missing_intervals = []

for i in range(0, 828, 4):
    try:
        start_index = i
        stop_index = start_index + 4
        song_df = get_songs(song_dict[start_index:stop_index])
        song_df.to_pickle(f'Spotify/new_approach/{start_index}_to_{stop_index}_of_song_dict.pkl')
        sleep(0.5);
    except:
        missing = [start_index, stop_index]
        list_of_missing_intervals.append(missing)
    sleep(1);
    #break
    #start_index = i
    

4 AM
It's A Vibe
PROUD
Bigger Than You (feat. Drake & Quavo)
Bank Account
Famous
My Choppa Hate Niggas
No Heart
X (feat. Future)
Ghostface Killers
Mad Stalkers
Rap Saved Me
Want You Back
Youngblood
PRBLMS
BILLY
GOTTI
GUMMO
KOODA
RONDO
TATI
KEKE
Beast Mode (feat. PnB Rock & YoungBoy Never Broke Again)
Drowning (feat. Kodak Black)
No Promises
Say A'
Undefeated (feat. 21 Savage)
Plain Jane
A$AP Forever
Praise The Lord (Da Shine)
Weak
Tennessee Rain - The Voice Performance
Water Under the Bridge
Growing Pains
How Far I'll Go - Alessia Cara Version
Scars To Your Beautiful
No Roots
Caroline
Fresh Eyes
It's the Most Wonderful Time of the Year
Beauty and the Beast
Everyday
no tears left to cry
Side To Side
the light is coming (feat. Nicki Minaj)
How Far I'll Go
Wake Me Up
Rolex
DDU-DU DDU-DU
DNA
FAKE LOVE
MIC Drop (feat. Desiigner) [Steve Aoki Remix]
Don't Go Breaking My Heart
Zombie
Mine
I Got You
Meant to Be (feat. Florida Georgia Line)
Mayores
Sin Pijama
Gucci Flip Flops (feat. Lil Yachty)


Despacito - Remix
Échame La Culpa
Fast
Light It Up
Most People Are Good
Sunrise, Sunburn, Sunset
Beautiful Crazy
Hurricane
Must've Never Met You
One Number Away
When It Rains It Pours
Lights Down Low
At My Best (feat. Hailee Steinfeld)
Home (with Machine Gun Kelly, X Ambassadors & Bebe Rexha)
Glorious (feat. Skylar Grey)
Good Old Days (feat. Kesha)
Know No Better
Run Up
Felices los 4
80s Mercedes
Dear Hate
I Could Use a Love Song
All I Want for Christmas Is You
I Don't Wanna Cry
Down
Cold (feat. Future)
Don't Wanna Know (feat. Kendrick Lamar)
Girls Like You (feat. Cardi B)
Wait
What Lovers Do (feat. SZA)
Alone
FRIENDS
Silence
Ocean (feat. Khalid)
Scared to Be Lonely
Famous
1942 Flows
Fuck That Check Up (feat. Lil Uzi Vert)
Heavy Heart
Issues
We Ball (feat. Young Thug)
Whatever You Need (feat. Chris Brown & Ty Dolla $ign)
Wins & Losses
NO EXCUSES
No Complaints
Think A Little Less
Drinkin' Problem
Auto Pilot
BBO (Bad Bitches Only)
Bad and Boujee (feat. Lil Uzi Vert)
CC
Call Casting
Cultu

In [238]:
file_paths = glob.glob('Spotify/new_approach/*_dict.pkl')
completed_df = pd.DataFrame()
for file in file_paths[:]:
    #print(file)
    df = pd.read_pickle(file)
    #print(df)
    completed_df = completed_df.append(df)
    #df.close()
    #break
    
completed_df.to_pickle('Spotify/new_approach/completed_spotify_files.pkl')
#completed_df = pd.read_csv('spotify.csv')
completed_df.drop(0, inplace = True, errors = 'ignore')
completed_df
#completed_df.to_csv('spotify.csv')

,acousticness,analysis_url,artist,danceability,duration_ms,energy,genre,id,instrumentalness,key,...,popularity,song,speechiness,tempo,time_signature,track_href,type,uri,valence,0
1,0.25900,https://api.spotify.com/v1/audio-analysis/5u64...,Lorde,0.563,214255.0,0.548,"[art pop, dance pop, electropop, metropopolis,...",5u64RlrnvvbtNh6R0EMh6o,0.000000,0.0,...,75.0,Homemade Dynamite - REMIX,0.1450,106.705,4.0,https://api.spotify.com/v1/tracks/5u64Rlrnvvbt...,audio_features,spotify:track:5u64RlrnvvbtNh6R0EMh6o,0.1710,NaN
2,0.91500,https://api.spotify.com/v1/audio-analysis/4iSN...,Lorde,0.608,171728.0,0.214,"[art pop, dance pop, electropop, metropopolis,...",4iSN72L6xCM7vQZR2I9Suz,0.000000,10.0,...,75.0,Liability,0.1230,75.751,4.0,https://api.spotify.com/v1/tracks/4iSN72L6xCM7...,audio_features,spotify:track:4iSN72L6xCM7vQZR2I9Suz,0.3310,NaN
3,0.42300,https://api.spotify.com/v1/audio-analysis/0Gl5...,Various Artists,0.277,207787.0,0.394,[],0Gl5s8IhMmQE5YQwM8Qx1J,0.000000,8.0,...,0.0,Never Enough,0.0336,86.777,3.0,https://api.spotify.com/v1/tracks/0Gl5s8IhMmQE...,audio_features,spotify:track:0Gl5s8IhMmQE5YQwM8Qx1J,0.2270,NaN
1,0.07770,https://api.spotify.com/v1/audio-analysis/75Zv...,Post Malone,0.573,223347.0,0.543,"[pop, rap]",75ZvA4QfFiZvzhj2xkaWAh,0.000000,8.0,...,94.0,I Fall Apart,0.0398,143.916,4.0,https://api.spotify.com/v1/tracks/75ZvA4QfFiZv...,audio_features,spotify:track:75ZvA4QfFiZvzhj2xkaWAh,0.3080,NaN
2,0.75100,https://api.spotify.com/v1/audio-analysis/0R6J...,Post Malone,0.341,112013.0,0.495,"[pop, rap]",0R6Jlsq25eZB3Wftpxi3oF,0.462000,0.0,...,94.0,Jonestown (Interlude),0.0446,78.395,4.0,https://api.spotify.com/v1/tracks/0R6Jlsq25eZB...,audio_features,spotify:track:0R6Jlsq25eZB3Wftpxi3oF,0.2520,NaN
3,0.35400,https://api.spotify.com/v1/audio-analysis/4qWC...,Post Malone,0.645,228227.0,0.657,"[pop, rap]",4qWCFTuNnmGNdY4egtb4ZY,0.000005,9.0,...,94.0,Otherside,0.0334,104.931,4.0,https://api.spotify.com/v1/tracks/4qWCFTuNnmGN...,audio_features,spotify:track:4qWCFTuNnmGNdY4egtb4ZY,0.1360,NaN
1,0.53600,https://api.spotify.com/v1/audio-analysis/2Rtt...,Ed Sheeran,0.522,207520.0,0.385,[pop],2RttW7RAu5nOAfq6YFvApB,0.000000,0.0,...,89.0,Happier,0.0288,89.792,4.0,https://api.spotify.com/v1/tracks/2RttW7RAu5nO...,audio_features,spotify:track:2RttW7RAu5nOAfq6YFvApB,0.2360,NaN
2,0.71900,https://api.spotify.com/v1/audio-analysis/2dfH...,Ed Sheeran,0.604,248413.0,0.366,[pop],2dfHh7ECGxfNqZTQno09Vk,0.000000,7.0,...,89.0,Hearts Don't Break Around Here,0.0267,105.177,3.0,https://api.spotify.com/v1/tracks/2dfHh7ECGxfN...,audio_features,spotify:track:2dfHh7ECGxfNqZTQno09Vk,0.1810,NaN
3,0.42400,https://api.spotify.com/v1/audio-analysis/6PJ8...,Ed Sheeran,0.617,280533.0,0.439,[pop],6PJ8FF6UR8FZXfEvpHkIVN,0.000000,9.0,...,89.0,How Would You Feel (Paean),0.0269,139.979,4.0,https://api.spotify.com/v1/tracks/6PJ8FF6UR8FZ...,audio_features,spotify:track:6PJ8FF6UR8FZXfEvpHkIVN,0.2420,NaN
1,0.02160,https://api.spotify.com/v1/audio-analysis/7Ckb...,XXXTENTACION,0.873,115194.0,0.663,[],7CkbRn6S0itJqcqn88AtSV,0.000356,1.0,...,95.0,going down!,0.1500,135.499,4.0,https://api.spotify.com/v1/tracks/7CkbRn6S0itJ...,audio_features,spotify:track:7CkbRn6S0itJqcqn88AtSV,0.5250,NaN


In [208]:
#completed_df.set_index('artist', inplace = True)
#completed_df.sort_values('artist', inplace = True)
completed_df.drop(['analysis_url', 'id', 'track_href', 'type', 'uri', 'index', '0'],axis =1, errors = 'ignore', inplace = True)
#completed_df.reset_index(inplace = True)
#completed_df.artist = completed_df.artist.str.strip()
#completed_df.song = completed_df.song.str.strip()

#completed_df.set_index(['artist', 'song'], inplace = True)
#completed_df.to_csv('spotify.csv')
completed_df

,acousticness,artist,danceability,duration_ms,energy,genre,instrumentalness,key,liveness,loudness,mode,popularity,song,speechiness,tempo,time_signature,valence,0
1,0.25900,Lorde,0.563,214255.0,0.548,"[art pop, dance pop, electropop, metropopolis,...",0.000000,0.0,0.1540,-5.126,0.0,75.0,Homemade Dynamite - REMIX,0.1450,106.705,4.0,0.1710,NaN
2,0.91500,Lorde,0.608,171728.0,0.214,"[art pop, dance pop, electropop, metropopolis,...",0.000000,10.0,0.0962,-11.161,0.0,75.0,Liability,0.1230,75.751,4.0,0.3310,NaN
3,0.42300,Various Artists,0.277,207787.0,0.394,[],0.000000,8.0,0.0952,-8.903,1.0,0.0,Never Enough,0.0336,86.777,3.0,0.2270,NaN
1,0.07770,Post Malone,0.573,223347.0,0.543,"[pop, rap]",0.000000,8.0,0.1990,-5.357,0.0,94.0,I Fall Apart,0.0398,143.916,4.0,0.3080,NaN
2,0.75100,Post Malone,0.341,112013.0,0.495,"[pop, rap]",0.462000,0.0,0.2610,-10.251,0.0,94.0,Jonestown (Interlude),0.0446,78.395,4.0,0.2520,NaN
3,0.35400,Post Malone,0.645,228227.0,0.657,"[pop, rap]",0.000005,9.0,0.2130,-5.688,0.0,94.0,Otherside,0.0334,104.931,4.0,0.1360,NaN
1,0.53600,Ed Sheeran,0.522,207520.0,0.385,[pop],0.000000,0.0,0.1350,-7.355,1.0,89.0,Happier,0.0288,89.792,4.0,0.2360,NaN
2,0.71900,Ed Sheeran,0.604,248413.0,0.366,[pop],0.000000,7.0,0.1620,-7.881,1.0,89.0,Hearts Don't Break Around Here,0.0267,105.177,3.0,0.1810,NaN
3,0.42400,Ed Sheeran,0.617,280533.0,0.439,[pop],0.000000,9.0,0.1270,-5.630,1.0,89.0,How Would You Feel (Paean),0.0269,139.979,4.0,0.2420,NaN
1,0.02160,XXXTENTACION,0.873,115194.0,0.663,[],0.000356,1.0,0.0867,-4.086,1.0,95.0,going down!,0.1500,135.499,4.0,0.5250,NaN


In [235]:
spotify = pd.read_csv('spotify.csv')
spotify.song = spotify.song.str.lower()
spotify.artist = spotify.artist.str.lower()
spotify.song = spotify.song.str.strip()
spotify.artist = spotify.artist.str.strip()
spotify

,artist,song,index,acousticness,danceability,duration_ms,energy,genre,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,valence,0
0,2 chainz,bigger than you (feat. drake & quavo),0,0.24700,0.888,225893.0,0.515,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.000000,1.0,0.4460,-6.246,1.0,83.0,0.2710,141.009,4.0,0.2640,NaN
1,2 chainz,proud,2,0.73800,0.781,234667.0,0.810,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.000000,10.0,0.1440,-4.301,0.0,83.0,0.3560,173.949,4.0,0.9080,NaN
2,2 chainz,it's a vibe,1,0.02630,0.841,210200.0,0.516,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.000720,11.0,0.1110,-7.461,0.0,83.0,0.1180,145.995,4.0,0.5560,NaN
3,2 chainz,4 am,0,0.12200,0.796,255560.0,0.513,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",0.000000,6.0,0.1500,-7.140,0.0,83.0,0.4520,75.018,4.0,0.2350,NaN
4,21 savage,mad stalkers,3,0.14700,0.900,202667.0,0.365,"['pop', 'rap', 'trap music']",0.185000,10.0,0.1790,-6.395,0.0,84.0,0.2350,135.003,4.0,0.1360,NaN
5,21 savage,ric flair drip (& metro boomin),2,0.14900,0.880,172800.0,0.428,"['pop', 'rap', 'trap music']",0.000051,9.0,0.1140,-8.280,1.0,84.0,0.2060,100.007,4.0,0.3330,NaN
6,21 savage,nightmare,1,0.44200,0.739,147987.0,0.405,"['pop', 'rap', 'trap music']",0.211000,0.0,0.1580,-10.897,1.0,84.0,0.0852,157.090,4.0,0.1270,NaN
7,21 savage,famous,2,0.72300,0.779,234280.0,0.597,"['pop', 'rap', 'trap music']",0.000045,9.0,0.0508,-9.652,0.0,84.0,0.4340,173.986,4.0,0.6460,NaN
8,21 savage,my choppa hate niggas,3,0.09740,0.862,148640.0,0.274,"['pop', 'rap', 'trap music']",0.185000,0.0,0.1830,-5.818,0.0,84.0,0.4840,77.493,4.0,0.1460,NaN
9,21 savage,ghostface killers,2,0.33100,0.782,268933.0,0.436,"['pop', 'rap', 'trap music']",0.000013,2.0,0.3420,-7.033,1.0,84.0,0.1550,82.994,4.0,0.2430,NaN


In [224]:
#billboards.reset_index(inplace = True)
print(billboards.columns)
billboards.drop(['level_0', 'index'], inplace = True, errors = 'ignore')
print(billboards.columns)
#billboards.artist = billboards.artist.str.strip()
#billboards.song = billboards.song.str.strip()
billboards.song = billboards.song.str.lower()
billboards.artist = billboards.artist.str.lower()
billboards.song = billboards.song.str.strip()
billboards.artist = billboards.artist.str.strip()

Index(['level_0', 'index', 'artist', 'song', 'num_weeks'], dtype='object')
Index(['level_0', 'index', 'artist', 'song', 'num_weeks'], dtype='object')


In [232]:
merged = billboards.merge(spotify, on = ['song', 'artist'], how = 'right')
merged#[merged.song == 'PROUD']
merged.sort_values('artist').to_csv('spotify_fix_me.csv')
merged = pd.read_csv('spotify_fixed.csv')

In [233]:
merged

,Unnamed: 0,artist,song,num_weeks,index_y,acousticness,danceability,duration_ms,energy,genre,...,key,liveness,loudness,mode,popularity,speechiness,tempo,time_signature,valence,0
0,0,2 chainz,4:00 AM,12.0,0,0.12200,0.796,255560.0,0.513,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",...,6.0,0.1500,-7.140,0.0,83.0,0.4520,75.018,4.0,0.2350,NaN
1,2,2 chainz,proud,2.0,2,0.73800,0.781,234667.0,0.810,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",...,10.0,0.1440,-4.301,0.0,83.0,0.3560,173.949,4.0,0.9080,NaN
2,586,2 chainz,bigger than you (feat. drake & quavo),2.0,0,0.24700,0.888,225893.0,0.515,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",...,1.0,0.4460,-6.246,1.0,83.0,0.2710,141.009,4.0,0.2640,NaN
3,1,2 chainz,it's a vibe,20.0,1,0.02630,0.841,210200.0,0.516,"['hip hop', 'pop', 'pop rap', 'rap', 'southern...",...,11.0,0.1110,-7.461,0.0,83.0,0.1180,145.995,4.0,0.5560,NaN
4,3,21 savage,bank account,28.0,1,0.01510,0.884,220307.0,0.346,"['pop', 'rap', 'trap music']",...,8.0,0.0871,-8.228,0.0,84.0,0.3510,75.016,4.0,0.3760,NaN
5,589,21 savage,nightmare,1.0,1,0.44200,0.739,147987.0,0.405,"['pop', 'rap', 'trap music']",...,0.0,0.1580,-10.897,1.0,84.0,0.0852,157.090,4.0,0.1270,NaN
6,590,21 savage,my choppa hate niggas,2.0,3,0.09740,0.862,148640.0,0.274,"['pop', 'rap', 'trap music']",...,0.0,0.1830,-5.818,0.0,84.0,0.4840,77.493,4.0,0.1460,NaN
7,591,21 savage,ghostface killers,7.0,2,0.33100,0.782,268933.0,0.436,"['pop', 'rap', 'trap music']",...,2.0,0.3420,-7.033,1.0,84.0,0.1550,82.994,4.0,0.2430,NaN
8,592,21 savage,x (feat. future),1.0,1,0.37300,0.835,258880.0,0.413,"['pop', 'rap', 'trap music']",...,1.0,0.1010,-9.810,1.0,84.0,0.3960,149.004,4.0,0.1110,NaN
9,593,21 savage,rap saved me,3.0,0,0.00525,0.845,257600.0,0.399,"['pop', 'rap', 'trap music']",...,1.0,0.1020,-7.509,1.0,84.0,0.3090,143.049,4.0,0.1480,NaN


In [128]:
completed_df.reset_index(inplace = True)
completed_df.sort_values(by = 'artist', inplace = True)
completed_df.set_index(['artist', 'song'], drop = False, inplace = True)
#completed_df.drop(, inplace = True)
#completed_df.to_csv('spotify.csv')
#billboards.set_index(['artist', 'song'], inplace = True)
billboards
#Counter(billboards.song)
#completed_df.merge(billboards)
#completed_df

ValueError: cannot insert song, already exists

In [179]:
completed_df.reset_index(inplace = True, drop = True)
print(completed_df)
print(completed_df.info())
ompleted_df.artist
# = completed_df.artist.str.strip()
completed_df.song = completed_df.song.str.strip()
#billboards.reset_index(inplace = True)
print(billboards)
billboards.drop('index', axis = 1, inplace = True, errors = 'ignore')
billboards.artist = billboards.artist.str.strip()
billboards.song = billboards.song.str.strip()

billboards.merge(completed_df)

     artist  song  index  acousticness  danceability  duration_ms  energy  \
0       NaN   NaN      0       0.24700         0.888     225893.0   0.515   
1       NaN   NaN      2       0.73800         0.781     234667.0   0.810   
2       NaN   NaN      1       0.02630         0.841     210200.0   0.516   
3       NaN   NaN      0       0.12200         0.796     255560.0   0.513   
4       NaN   NaN      1       0.01510         0.884     220307.0   0.346   
5       NaN   NaN      0       0.13600         0.906     235040.0   0.444   
6       NaN   NaN      1       0.37300         0.835     258880.0   0.413   
7       NaN   NaN      2       0.33100         0.782     268933.0   0.436   
8       NaN   NaN      0       0.00525         0.845     257600.0   0.399   
9       NaN   NaN      2       0.72300         0.779     234280.0   0.597   
10      NaN   NaN      1       0.44200         0.739     147987.0   0.405   
11      NaN   NaN      2       0.14900         0.880     172800.0   0.428   

NameError: name 'ompleted_df' is not defined

In [164]:
print(billboards.str.string)

AttributeError: 'DataFrame' object has no attribute 'str'

In [137]:
billboards.reset_index(inplace = False)
completed_df.join(billboards)

TypeError: '<' not supported between instances of 'float' and 'str'

In [121]:
lst = [completed_df, billboards]
yay = pd.concat(lst, axis = 1)

Exception: cannot handle a non-unique multi-index!

In [76]:
billboards = pd.read_pickle('828_songs.pkl')
#print(len(completed_df))
billboards.reset_index(inplace=True)
#print(billboards.columns)
billboards.rename(columns = {0: 'num_weeks', "Artist": 'artist', 'Song':'song'}, inplace = True)
#billboards.set_index(['artist', 'song'], inplace = True)
#completed_df['num_weeks'] = billboards['num_weeks']
merged = billboards.merge(completed_df)
merged
#billboards.join(completed_df)

MergeError: No common columns to perform merge on

In [ ]:
#completed_df.set_index(['artist', 'song'], inplace = True)
completed_df.drop_duplicate()

In [ ]:
completed_df = pd.read_pickle('Spotify/new_approach/completed_spotify_files.pkl')
recent_songs = pd.read_pickle('recent_songs_to_spotify.pkl')
completed_df.columns
recent_songs.rename(columns = {'Artist': 'artist', "Song":'song', 0: 'num_weeks'}, inplace = True)
completed_df.song.sort_values()

#need to grab the num_weeks from the merged data set and add it back to this data set

In [ ]:
completed_df['num_weeks'] = recent_songs['num_weeks']
#complete.to_pickle('recent_songs_spotify.pkl')
complete

In [ ]:
#recent_songs = recent_songs.rename(columns = {'Artist': 'artist', 'Song': 'song'})
#print(recent_songs)
#print(recent_songs)
#print(outer_merged_df)
#recent_songs.set_index(['artist', 'song'], inplace = True)
#print(recent_songs)
#get_num_weeks_df = recent_songs.set_index(['artist', 'song']).num_weeks
#print(get_num_weeks_df)
#get_num_weeks_df = get_num_weeks_df.rename(columns = {'Artist': 'artist', 'Song': 'song'})
#print(get_num_weeks_df)
completed_wo_num_weeks = completed_df.set_index(['artist', 'song'])
#print(completed_wo_num_weeks)
joined = completed_wo_num_weeks.join(recent_songs, how = "inner")

#joined.num_weeks.fillna(0, inplace = True)

joined
#sns.pairplot(joined)

#Counter(joined.num_weeks)

In [ ]:
joined
joined.reset_index(inplace = True)
joined.to_pickle('recent_songs_spotify.pkl')
joined

In [ ]:
# import spotipy
# spotify = spotipy.Spotify()

# results = sp.search(q="track:baby artist:justin bieber", type='track')
# a = results['tracks']['items'][0]['id']
# #list(a.keys())
# #print(a)

# d = sp.audio_analysis(a)['track']['tempo']
# print(d)
# #list(d.keys())

# #print((results['artists']['items'][0]))
song_df
print('done')

## I need to load the completed df to create a new column with the track id 

In [ ]:
song_df

## I need to find a way to add the song's num weeks back to the data frame

In [ ]:
unfound = pd.read_pickle('unfound_songs.pkl')
unfound